In [8]:
language = "pt"

In [15]:
#Gravação de audio com Python e JS
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_time_ms = sec * 1000
  js_result = output.eval_js(f'record({js_time_ms})')
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
      f.write(audio)
  return f"/content/{file_name}"

print("Gravando...\n")
record_file = record()
display(Audio(record_file, autoplay=True))

Gravando...



<IPython.core.display.Javascript object>

In [10]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
#Reconhecimento de fala Whisper
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 bom dia bom dia por dia por dia
